In [ ]:
#楊桃網完整版(20753筆)-楊桃 http://recipe.ytower.com.tw/ 軒哥
import requests
from bs4 import BeautifulSoup
rs = requests.session()
rs_get = rs.get('http://recipe.ytower.com.tw/Channel/MultiSearchRecipes/?catalogs=%E4%B8%AD%E5%BC%8F%E6%96%99%E7%90%86%2C%E8%A5%BF%E5%BC%8F%E6%96%99%E7%90%86%2C%E6%97%A5%E5%BC%8F%E6%96%99%E7%90%86%2C%E9%9F%93%E5%BC%8F%E6%96%99%E7%90%86%2C%E5%8D%97%E6%B4%8B%E6%96%99%E7%90%86%2C%E5%85%B6%E4%BB%96%E6%96%99%E7%90%86')
soup = BeautifulSoup(rs_get.text)
counts = float(soup.select('.rcp_tab03')[0].text[3:-1]) #取出總筆數
import math
pagenum = int(math.ceil(float(counts) / 10)) #計算多少頁

f ='http://recipe.ytower.com.tw/Channel/MultiSearchRecipes/?catalogs=%E4%B8%AD%E5%BC%8F%E6%96%99%E7%90%86%2C%E8%A5%BF%E5%BC%8F%E6%96%99%E7%90%86%2C%E6%97%A5%E5%BC%8F%E6%96%99%E7%90%86%2C%E9%9F%93%E5%BC%8F%E6%96%99%E7%90%86%2C%E5%8D%97%E6%B4%8B%E6%96%99%E7%90%86%2C%E5%85%B6%E4%BB%96%E6%96%99%E7%90%86&page={0}'

ytower_file = open("y_tower.txt", 'w')
for page in range(1,pagenum+1):
    bid_list = rs.get(f.format(page)) 
    bid_response = bid_list.text.encode('utf8')
    soup = BeautifulSoup(bid_response)
    for i in soup.select('.left_box'):
        tag = i.select('a')[0]
        href ='http://recipe.ytower.com.tw' + tag['href'] #取得完整網址
        ytower_file.write(href + "\n") #寫入txt檔案中
ytower_file.close()

bid_readfile = open("y_tower.txt", 'r')
print len([line for line in bid_readfile.readlines()]) #驗證txt檔案筆數是否正確
bid_readfile.close()

ytower_file = open('y_tower.txt', 'r')
test1 = open('YTower_ingregients.txt','w')
test2 = open('Ytower_practice.txt','w')
for line in ytower_file.readlines():
    res = requests.get(line.strip())
    soup = BeautifulSoup(res.text)
    title = soup.select('.rcp_detailbox h3')[0].text.encode('utf-8') #取出標題
    ingredients = soup.select('.rcp_materialbox li') #取出食材
    practice = soup.select('.rcp_practicebox .txt') #取出作法
    a=[]
    a.append(title) #把標題append進list中
    for item  in ingredients[:-1]:
        a.append(item.select('.object')[0].text.encode('utf-8')) 
        a.append(item.select('.unit')[0].text.encode('utf-8')) #把食材append進list中
    b=[]
    b.append(title)
    for step in practice:
        b.append(step.text.encode('utf-8')) #把作法append進list中
    for i in a:
        test1.write(i+',') #標題和食材寫入YTower_ingregients.txt檔案
    test1.write('\n')    
    for i in b:
        test2.write(i+',') #標題和作法寫入Ytower_practice.txt檔案
    test2.write('\n')   

test1.close()
test2.close()

In [ ]:
#全聯食譜859筆-全聯食譜 http://cook.pxmart.com.tw/search.aspx May
import requests
from bs4 import BeautifulSoup
import re
res = requests.get('http://cook.pxmart.com.tw/search.aspx')
page_format = "http://cook.pxmart.com.tw/search.aspx?cp=&ck=&s_o=&p_o=&d_o=&page=%d"

recipe_file = open("recipe_list.txt", 'w')
for page in range(0,54): 
    recipe_list = requests.get(page_format%(page)) 
    recipe_response = recipe_list.text
    soup = BeautifulSoup(recipe_response)
    recipe_table = soup.select("#recipe")[0]
    recipe_rows = recipe_table.select('.recipe_box')
    for recipe_row in recipe_rows:
        links = [tag['href'] for tag in recipe_row.select('a')] 
        link_href = "http://cook.pxmart.com.tw/" + links[0]  
        recipe_file.write(link_href + "\n") 
recipe_file.close()

#把每頁菜單的食材和步驟寫入pxmart_ingredients.txt和pxmart_steps.txt
recipe_file = open("recipe_list.txt", 'r')
pxmart_ingredients = open('pxmart_ingredients.txt','w')
pxmart_steps = open('pxmart_steps.txt','w')
for line in recipe_file.readlines(): 
    pagelink = line.strip() 
    request_get = requests.get(pagelink) 
    response = request_get.text
    soup = BeautifulSoup(response) 

    ingredientsList = []
    title = soup.select(".recipe_title")[0].text.encode('utf-8').strip()
    ingredientsList.append(title) #1.放title到ingredientsList
    ingredients = soup.select(".recipe_food .recipe_ine")
    for i in ingredients:  #2.放ingredients到ingredientsList      
        if len(i.select('.recipe_left_name')) >0:
            ingredientsList.append(i.select('.recipe_left_name')[0].text.encode('utf-8').strip()[6:]) #刪掉開頭的“全聯".
        if len(i.select('.recipe_left_name_nopic')) >0:
            ingredientsList.append(i.select('.recipe_left_name_nopic')[0].text.encode('utf-8').strip()[6:]) #沒有圖片的run this..
        ingredientsList.append(i.select('.recipe_left_unit')[0].text.encode('utf-8').strip()) #量

    seasoning = soup.select(".recipe_seasoning .recipe_ine")
    for i in seasoning: #3.放seasoning到ingredientsList      
        ingredientsList.append(i.select('.recipe_left_name_nopic_flavour')[0].text.encode('utf-8').strip())
        ingredientsList.append(i.select('.recipe_left_unit_flavour')[0].text.encode('utf-8').strip())
        
    stepsList = []
    stepsList.append(title) #1.放title到stepsList
    steps = soup.select(".recipe_step_all")
    for i in steps: #2.放steps到stepsList
        if len(i.select('.recipe_step_left')) >0:
            stepsList.append(i.select('.recipe_step_left .recipe_step_w')[0].text.encode('utf-8').strip())
        if len(i.select('.recipe_step_right')) >0:
            stepsList.append(i.select('.recipe_step_right .recipe_step_w')[0].text.encode('utf-8').strip())         

    
    for i in ingredientsList:
        pxmart_ingredients.write((i+','))
    pxmart_ingredients.write('\n')    
    for i in stepsList:
        pxmart_steps.write((i+','))
    pxmart_steps.write('\n')   
    
pxmart_steps.close()
pxmart_ingredients.close()
recipe_file.close()

In [ ]:
#我的美食DIY(2114筆)-我的美食DIY http://www.ttv.com.tw/cuisine/Search.aspx?lnk=tag&val=hotrecipe&page=1 老蘇
# -*- coding: utf8 -*-   
import string               
import requests
import re
from bs4 import BeautifulSoup
res = requests.get("http://www.ttv.com.tw/cuisine/Search.aspx?lnk=tag&val=hotrecipe&page=1")
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text.encode('utf-8'))
pagenum = int(soup.select('#ctl00_cph1_pagingResult_pnlPaging')[0].select('#ctl00_cph1_pagingResult_lblPageCount')[0].text.encode('utf-8').strip())
print pagenum
total=0

f='http://www.ttv.com.tw/cuisine/Search.aspx?lnk=tag&val=hotrecipe&page={0}'
ttv_file = open("ttv.txt", 'w')
test1 = open("ttv_materials.txt", 'w')
test2 = open("ttv_practice.txt", 'w')
for page in range(1,pagenum+1):
    res = requests.get(f.format(page))
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text.encode('utf-8'))
    for i in soup.select('.fcon'):
        tag = i.select('a')[0]['href']   
        href = 'http://www.ttv.com.tw/cuisine/'+tag        
        print href
        ttv_file.write(href + "\n") #寫入txt檔案中
        
        res = requests.get(href)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text.encode('utf-8'))
        #print soup
        
        main = soup.select('.style15')[0].text
        print 'name:'+ main +'\n'
        
        materials= soup.select('ul')[0].select('.style11')[0].text.encode('utf-8').strip()
        print 'materials:'+ materials +'\n'
        test1.write(str(main.encode('utf-8'))+',')
        test1.write(materials + "\n")
        
        practice = soup.select('ul')[0].select('.style11')[1].text.encode('utf-8').strip()
        practice_1=re.sub("\d\.", " ", practice)     #消滅所有"數字."
        practice_2=string.join(practice_1.split(' '),',')#"空白"轉換成","
        print 'practice:'+ practice_2 +'\n'
        test2.write(str(main.encode('utf-8')))
        test2.write(practice_2 + "\n")
        
        total+=1
print total
ttv_file.close()
test1.close()
test2.close()

In [ ]:
#自由時報(2959筆)-自由時報 http://week.ltn.com.tw/CookBook/index.html 7~3243 軒哥
import requests
from bs4 import BeautifulSoup

page_format = "http://week.ltn.com.tw/CookBook/DC%d.html"

ltn_href = open('LTN_lists.txt','w')

for page in range(7,3274):
    ltn_href.write(page_format%(page))
    ltn_href.write('\n')

ltn_href.close()   

import requests
from bs4 import BeautifulSoup

ltn_file = open("LTN_lists.txt", 'r')
test1 = open('LTN_ingregients.txt','w')
test2 = open('LTN_practice.txt','w')
#test3 = open('LTN_others.txt','w')

for line in ltn_file.readlines():
    try:
        pagelink = line.strip()
        print pagelink
        res = requests.get(pagelink)
        soup = BeautifulSoup(res.text)

        #cat1 = soup.select('#ItemSubContent_uc_SiteMap_Link_LinkButton_TypeClass_a')[0].text
        #cat2 = soup.select('#ItemSubContent_uc_SiteMap_Link_LinkButton_TypeClass_b')[0].text
        #remark =  soup.select('#MainContent_uc_MainContent_uc_MainContent1_Lbl_Remarks')[0].text
        title = soup.select('.HL_latest_new_hover')[0].text
        ingredients = soup.select('#MainContent_uc_MainContent_uc_MainContent1_Lbl_f_material')[0].text

        steps=[]
        steps.append(title)

        step = 0
        while True:
            try:
                a = soup.select('#MainContent_uc_MainContent_uc_MainContent1_uc_StepByStep_ListView1_tblContacts')[0].select('#MainContent_uc_MainContent_uc_MainContent1_uc_StepByStep_ListView1_ctrl'+str(step)+'_LblConent_'+str(step))
                steps.append(a[0].text)
                #print steps[step]
                step+=1
            except IndexError:
                break;

        test1.write(title.encode('utf-8') +',' + ingredients.encode('utf-8'))
        test1.write('\n')

        for i in steps:
            test2.write(i.encode('utf-8')+ ',')
        test2.write('\n')

        #test3.write(title.encode('utf-8') +',' + remark.encode('utf-8'))
        #test3.write('\n')
    except:
        pass


#test3.close()
test2.close()
test1.close()
ltn_file.close()


In [ ]:
#菲姐-全部爬出來, 內容很不一致, 但仍建議用此方法爬出,在手工整理 ＃363筆
import requests
from bs4 import BeautifulSoup
page_format = "http://www.italian-lifestore.com.tw/recipe.php?SearchRecipeVideoCateID=&SearchKey=&SearchType1=&SearchType2=&Page=%d"

recipe_file = open("sis_recipe_list.txt", 'w')
for page in range(0,38): 
    recipe_list = requests.get(page_format%(page)) 
    recipe_response = recipe_list.text
    soup = BeautifulSoup(recipe_response)
    recipe_table = soup.select(".food")[0]
    recipe_rows = recipe_table.select('.clearfix')
    for recipe_row in recipe_rows:
        links = [tag['href'] for tag in recipe_row.select('a')] 
        link_href = "http://www.italian-lifestore.com.tw/" + links[0] 
        recipe_file.write(link_href + "\n") 
recipe_file.close()

#把每頁菜單的食材和步驟寫入
recipe_file = open("sis_recipe_list.txt", 'r')
for line in recipe_file.readlines(): 
    pagelink = line.strip() 
    request_get = requests.get(pagelink) 
    request_get.encoding = 'utf-8'
    soup = BeautifulSoup(request_get.text.encode('utf-8')) 
    try:
        sis1_ingredients = open('sis1_ingredients.txt','a')
        ingredientsList = [] 
        ingredients = soup.select(".recipe-tBox")[0].text.strip()
        a = ''.join(ingredients.split('-'))
        b = ','.join(a.split())
        ingredientsList.append(b)

        for i in ingredientsList:
            sis1_ingredients.write((i+',').encode('utf-8')) 
        sis1_ingredients.write('\n')    
    except:
        pass
sis1_ingredients.close()
recipe_file.close()

#How to replace “烹飪步驟” with the content in front of first comma? = Title name.
#break sentences to two parts by "烹飪步驟,"
in_file = open('sis1_ingredients.txt','r')
out_file_ingredients = open('sis3_title_ingredients.txt','w')
out_file_steps = open('sis3_title_steps.txt','w')
ingredients = []
steps = []
for line in in_file.readlines():
    ingredient = line.split("烹飪步驟,")[0]
    ingredients.append(ingredient)
    title = line.split(",")[0]
    steps.append(title)
    step = line.split("烹飪步驟,")[1].strip()
    steps.append(step)
for i in ingredients:    
    out_file_ingredients.write(i+'\n')
for i in steps:
    out_file_steps.write(i+',')
out_file_steps.write('\n')
out_file_ingredients.close()
out_file_steps.close()
in_file.close()
#在手工整理一下ＸＤ

In [ ]:
#型男大主廚 - 2602筆
#Next- How to delete the rows which contain only one comma?
import requests
from bs4 import BeautifulSoup
import re
import string
page_format = "http://blog.iset.com.tw//chef/?p=%d"
recipe_file = open("iset_chef_list.txt", 'w')
for page in range(0,13000): 
    recipe_list = page_format%(page) 
    recipe_file.write(recipe_list + "\n")    
recipe_file.close()

#把每頁菜單的食材和步驟寫入iset_ingredients_steps.txt
recipe_file = open("iset_chef_list.txt", 'r')
iset_ingredients_steps = open('iset-chef_ingredients_steps.txt','w')
for line in recipe_file.readlines(): 
    pagelink = line.strip() 
    request_get = requests.get(pagelink)
    request_get.encoding = 'utf-8'
    response = request_get.text
    soup = BeautifulSoup(response) 
    try:
        if "Not Found" in str(soup.select("#inwrapper .title")[0]):
            pass

        else:
            ingredientsList = []
            ingredients = soup.select("#inwrapper div")[1].text.encode('utf-8').strip()
            if 'var addthis_pub' in ingredients:
                a = ingredients.split('var addthis_pub')[0]
                b = ','.join(a.split())
                ingredientsList.append(b)
            
            else:
                ingredientsList.append(ingredients)
                
            for i in ingredientsList:
                iset_ingredients_steps.write(i+',')
            iset_ingredients_steps.write('\n')     
        
    except:
        pass
iset_ingredients_steps.close()
recipe_file.close()

#skip duplicate
in_file = open('iset-chef_ingredients_steps.txt','r')
out_file = open('2-iset-chef_ingredients_steps.txt','w')
unique = set()
for line in in_file.readlines():
    if line in unique: 
        continue 
    else:
        unique.add(line)
        out_file.write(line)
in_file.close()
out_file.close()

#remove those rows which contain only one comma
in_file = open('2-iset-chef_ingredients_steps.txt','r')
out_file = open('3-iset-chef_ingredients_steps.txt','w')

x = []
for line in in_file.readlines():
    if len(line.split(',')) <= 2:
        continue
    else:
        x.append(line)
for i in x:
    out_file.write(i)
    
out_file.close()
in_file.close()

import re
in_file = open('3-iset-chef_ingredients_steps.txt','r')
ingredientsList_file = open('4-iset-chef_ingredients.txt','w')
stepsList_file = open('4-iset-chef_steps.txt','w')

def cut(line,x):
    ingredientsList = []
    stepsList = []
    i=0
    for i in range(0, x):
        if len(line.split(',食材：,',x)[i].rsplit(',', 1)[1].split('】')[1])>=2:
            title = line.split(',食材：,',x)[i].rsplit(',', 1)[1].split('】')[1]
        elif len(line.split(',食材：,',x)[i].rsplit(',', 1)[1].split('【')[0])>=2:
            title = line.split(',食材：,',x)[i].rsplit(',', 1)[1].split('【')[0]
        else:
            title = line.split(',食材：,',x)[i].rsplit(',', 1)[1]
        ingredientsList.append(title + ',')
        stepsList.append(title + ',')
        ingredients = line.split(',食材：,',x)[i+1].split(',作法：,')[0]
        steps = line.split(',食材：,',x)[i+1].split(',作法：,')[1].rsplit(',',1)[0]
        ingredientsList.append(ingredients+'\n')
        stepsList.append(steps+'\n')
        i += 1
    for k in ingredientsList:
        ingredientsList_file.write(k)
    for k in stepsList:
        stepsList_file.write(k)        
    return     
    
for line in in_file.readlines():
    for j in range (1,10):
        if len([m.start() for m in re.finditer(',食材：,', line)]) == j:
            try:
                cut(line,j)
            except:
                pass
    
ingredientsList_file.close()
stepsList_file.close()    
in_file.close()

#skip duplicate again
in_file = open('4-iset-chef_ingredients.txt','r')
out_file = open('5-iset-chef_ingredients.txt','w')
unique = set()
for line in in_file.readlines():
    if line in unique: 
        continue 
    else:
        unique.add(line)
        out_file.write(line)
in_file.close()
out_file.close()

#skip duplicate again
in_file = open('4-iset-chef_steps.txt','r')
out_file = open('5-iset-chef_steps.txt','w')
unique = set()
for line in in_file.readlines():
    if line in unique: 
        continue 
    else:
        unique.add(line)
        out_file.write(line)
in_file.close()
out_file.close()

#手工刪除約20筆dups

In [1]:
#行政院農委會197筆 

In [ ]:
#美食鳳味
import requests
from bs4 import BeautifulSoup
import re
import string
page_format = "http://blog.iset.com.tw/delicious/?p=%d"
recipe_file = open("iset_recipe_list.txt", 'w')
for page in range(1,15000): 
    recipe_list = page_format%(page) 
    recipe_file.write(recipe_list + "\n")    
recipe_file.close()

#把每頁菜單的食材和步驟寫入iset_ingredients_steps.txt
recipe_file = open("iset_recipe_list.txt", 'r')
iset_ingredients_steps = open('iset_ingredients_steps.txt','w')
for line in recipe_file.readlines(): 
    pagelink = line.strip() 
    request_get = requests.get(pagelink)
    request_get.encoding = 'utf-8'
    response = request_get.text
    soup = BeautifulSoup(response) 
    try:
        if "Not Found" in str(soup.select("#inwrapper .title")[0]):
            pass

        else:
            ingredientsList = []
            ingredients = soup.select("#inwrapper div")[1].text.encode('utf-8').strip()
            if 'var addthis_pub' in ingredients:
                a = ingredients.split('var addthis_pub')[0]
                b = ','.join(a.split())
                ingredientsList.append(b)
            
            else:
                ingredientsList.append(ingredients)
                
            for i in ingredientsList:
                iset_ingredients_steps.write(i+',')
            iset_ingredients_steps.write('\n')     
        
    except:
        pass
iset_ingredients_steps.close()
recipe_file.close()

#skip duplicate
in_file = open('iset_ingredients_steps.txt','r')
out_file = open('2-iset_ingredients_steps.txt','w')
unique = set()
for line in in_file:
    if line in unique: 
        continue 
    else:
        unique.add(line)
        out_file.write(line)
in_file.close()
out_file.close()

#Next- How to delete the rows which contain only 2 comma?
in_file = open('2-iset_ingredients_steps.txt','r')
out_file = open('3-iset_ingredients_steps.txt','w')

x = []
for line in in_file.readlines():
    if len(line.split(',')) <= 3:
        continue
    else:
        x.append(line)
for i in x:
    out_file.write(i)
    
in_file.close()
out_file.close()

in_file = open('3-iset_ingredients_steps.txt','r')
ingredientsList_file = open('4-iset_MeiFeng_ingredients.txt','w')
stepsList_file = open('4-iset_MeiFeng_steps.txt','w')

def cut(line,x):
    ingredientsList = []
    stepsList = []
    i=0
    for i in range(0, x):    
        title = line.split(',食材：,',x)[i].split(',', 3)[2]
        ingredientsList.append(title + ',')
        stepsList.append(title + ',')
        ingredients = line.split(',食材：,',x)[i+1].split(',作法：,')[0]
        steps = line.split(',食材：,',x)[i+1].split(',作法：,')[1].rsplit(',',1)[0]
        ingredientsList.append(ingredients+'\n')
        stepsList.append(steps+'\n')
        i += 1
    for k in ingredientsList:
        ingredientsList_file.write(k)
    for k in stepsList:
        stepsList_file.write(k)        
    return     
    
for line in in_file.readlines():
    for j in range (1,10):
        if len([m.start() for m in re.finditer(',食材：,', line)]) == j:
            try:
                cut(line,j)
            except:
                pass
    
ingredientsList_file.close()
stepsList_file.close()    
in_file.close()

import string
in_file = open('4-iset_MeiFeng_ingredients.txt','r')
out_file = open('5-iset_MeiFeng_ingredients.txt','w')
unique = []
for line in in_file.readlines():
    if line.find("作法：") == -1: 
        unique.append(line)
        out_file.write(line)
    else:
        pass
in_file.close()
out_file.close()

in_file = open('4-iset_MeiFeng_steps.txt','r')
out_file = open('5-iset_MeiFeng_steps.txt','w')
unique = []
for line in in_file.readlines():
    if line.find("作法：") == -1: 
        unique.append(line)
        out_file.write(line)
    else:
        pass
in_file.close()
out_file.close()